## 1 Hierarchical Indexing

___Hierarchical indexing___ is an important fearture of pandas that enables you to have multiple index levels on an axis.

Create a Series with a list of lists (or arrays) as the index, `MultiIndex`

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [8]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 2, 3, 1, 2, 3]])

In [9]:
data

a  1    1.505454
   2   -0.115230
   3    1.244119
b  1   -0.043838
   2   -0.480653
c  3    0.910750
   1   -0.256528
d  2    1.548743
   3   -1.431489
dtype: float64

In [10]:
data.index

MultiIndex(levels=[[u'a', u'b', u'c', u'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 2]])

With a hierarchically indexed object, so-called partial indexing is possible, enabling you to concisely select subsets of the data

In [11]:
data['b']

1   -0.043838
2   -0.480653
dtype: float64

In [12]:
data['b':'c']

b  1   -0.043838
   2   -0.480653
c  3    0.910750
   1   -0.256528
dtype: float64

In [13]:
data.loc[['b', 'd']]

b  1   -0.043838
   2   -0.480653
d  2    1.548743
   3   -1.431489
dtype: float64

In [14]:
data.loc[:, 2]

a   -0.115230
b   -0.480653
d    1.548743
dtype: float64

Rearrange the data into a DataFrame using its `unstack` method

In [16]:
data.unstack()

,1,2,3
a,1.505454,-0.115230,1.244119
b,-0.043838,-0.480653,NaN
c,-0.256528,NaN,0.910750
d,NaN,1.548743,-1.431489


The inverse operation of `unstack` is `stack`

In [17]:
data.unstack().stack()

a  1    1.505454
   2   -0.115230
   3    1.244119
b  1   -0.043838
   2   -0.480653
c  1   -0.256528
   3    0.910750
d  2    1.548743
   3   -1.431489
dtype: float64

With a DataFrame, either axis can have a hierarchical index

In [18]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), 
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']],
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

In [19]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names

In [20]:
frame.index.names = ['key1', 'key2']

In [21]:
frame.columns.names = ['state', 'color']

In [22]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Partial indexing of a DataFrame

In [23]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [24]:
frame.loc['a']

state  Ohio     Colorado
color Green Red    Green
key2                    
1         0   1        2
2         3   4        5

In [25]:
frame.loc['a', 'Ohio']

color,Green,Red
key2,,
1,0,1
2,3,4


A `MultiIndex` can be created by itself and then reused

In [28]:
pd.MultiIndex(levels=[['Ohio', 'Colorado'], ['Green', 'Red']],
               labels=[[0, 0, 1], [0, 1, 0]],
               names=['state', 'color'])

MultiIndex(levels=[[u'Ohio', u'Colorado'], [u'Green', u'Red']],
           labels=[[0, 0, 1], [0, 1, 0]],
           names=[u'state', u'color'])

In [29]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                          names=['state', 'color'])

MultiIndex(levels=[[u'Colorado', u'Ohio'], [u'Green', u'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=[u'state', u'color'])

### 1.1 Reordering and Sorting Levels

The `swaplevel` method takes two level numbers or names and return a new object with the levels interchanged

In [34]:
frame.swaplevel('key2', 'key1')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [36]:
frame.swaplevel('state', 'color', axis=1)

color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

`sort_index` sorts the data using only the labels in a single level.

Data selection performance is much better on hierarchically indexed objects if the index is lexicographically sorted starting with the outermost level

In [53]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

When sorting by a MultiIndex with `sort_values`, you need to contain tuple describing the column inside a list.

In [50]:
frame.sort_values([('Ohio', 'Green'), ('Colorado', 'Green')])

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### 1.2 Summary Statistic by Level

Many descriptive and summary statistics on DataFrame and Series have a `level` option in which you can specify the level you want to aggregate by on a particular axis

In [54]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [57]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 1.3 Indexing with a DataFrame's Columns

In [59]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                     'c': ['one', 'one', 'one', 'two', 'two','two', 'two'],
                     'd': [0,1,2,0,1,2,3]})

In [60]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame's `set_index` method will create a new DataFrame using one or more of it's columns as the index

In [62]:
frame2 = frame.set_index(['c', 'd'])

In [63]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

`reset_index` does the opposite of set_index. The hierarchical index levels are moved into the columns

In [67]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


`stack` and `unstack` are for pivot table, `set_index` and `reset_index` are used to set/reset columns to index

## 2 Combining and Merging Datasets

- `pandas.merge` connect rows in DataFrames based on one or more keys like SQL's join operation
- `pandas.concat` concatenates of "stack" together objects along on an axis
- The `combine_first` method of a DataFrame enables splicing together overlapping data to fill in missing values in one object with values from another

### 2.1 Database-Style DataFrame Joins

In [4]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

In [6]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [7]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [8]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [9]:
# If not specify which column to join,
# merge uses the same column name as keys
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [10]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


If the column names are different

In [11]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                     'data1': range(7)})

In [12]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                     'data2': range(3)})

In [13]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


`merge` does an 'inner' join by default.

In [14]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


To merge with multiple keys, pass a list of column names

In [19]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

In [20]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [21]:
pd.merge(left, right, how='outer', on=['key1', 'key2'])

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


When joining columns-to-columns, the indexs on the passed DataFrame objects are discarded

`merge` has a `suffixes` option for specifying strings to append to overlapping names in the left and right DataFrame objects

In [22]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [23]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 2.2 Merging on Index

Pass `left_index=True` and `right_index=True` to indicate that the index should be used as the merge key

In [24]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})

In [25]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [26]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [27]:
right1

,group_val
a,3.5
b,7.0


In [28]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


With hierarchically indexed data, joining on index is implicitly a multiple-key merge

In [29]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                      'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

In [30]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [31]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


DataFrame has `join` method for merging by index. It can also be used to combine together many DataFrame objects having the same or similar indexes but non-overlapping columns

In [34]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])

In [35]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [36]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame

In [37]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [39]:
right1

,group_val
a,3.5
b,7.0


In [41]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


For simple index-on-index merges, you can pass a list of DataFrames to `join` as an alternative to using the more general `concat` function described in the next section

In [42]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [43]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [44]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


### 2.3 Concatenating Along an Axis

NumPy's `concatenate` function can do concatenation, binding, or stacking

In [45]:
arr = np.arange(12).reshape((3, 4))

In [46]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [47]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

The `pd.concat` function can be used to concatenate Series and DataFrame with labeled axes

Concatenate three Series with no index overlap

In [48]:
s1 = pd.Series([0, 1], index=['a', 'b'])

In [49]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

In [50]:
s3 = pd.Series([5, 6], index=['f', 'g'])

In [53]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [55]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [56]:
s4 = pd.concat([s1, s3])

In [57]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [58]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [59]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


Specify the axes to be used on the other axes with `join_axes`

In [60]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


Create a hierarchical index on the concatenation axis.

In [70]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [71]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [72]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In the case of combining Series along axis=1, the keys become the DataFrame column headers

In [73]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


If a dict of objects is passed, the dict's keys will be used for the `keys` option

In [74]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])

In [75]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

In [76]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [77]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [78]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

`ignore_index=True`

In [79]:
df1

,a,b,c,d
0,1.699439,0.645029,1.672881,-1.581357
1,-0.085498,-0.693521,-0.967058,-1.224068
2,-1.739488,-0.448428,-1.685198,0.471997


In [80]:
df2

,b,d,a
0,-0.125650,0.568868,-3.821127
1,0.576525,2.358280,0.204947


In [81]:
pd.concat([df1, df2])

,a,b,c,d
0,1.699439,0.645029,1.672881,-1.581357
1,-0.085498,-0.693521,-0.967058,-1.224068
2,-1.739488,-0.448428,-1.685198,0.471997
0,-3.821127,-0.125650,NaN,0.568868
1,0.204947,0.576525,NaN,2.358280


In [84]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.699439,0.645029,1.672881,-1.581357
1,-0.085498,-0.693521,-0.967058,-1.224068
2,-1.739488,-0.448428,-1.685198,0.471997
3,-3.821127,-0.125650,NaN,0.568868
4,0.204947,0.576525,NaN,2.358280


### 2.4 Combining Data with Overlap

In [85]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [86]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [90]:
b[-1] = np.nan

In [91]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [92]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [93]:
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

Series has a `combine_first` method, which performs the equivalent of np.where operation, filling in missing values in one object with values from another

In [95]:
b.combine_first(a)

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [96]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

In [97]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [98]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [99]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [102]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 3 Reshaping and Pivoting

Basic operations for rearranging tabular data: `reshape` or `pivot`

### 3.1 Reshaping with Hierarchical Indexing

- `stack` "rotate" or pivots from the columns in the data to the rows
- `unstack` pivots from the rows into columns

In [105]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))

In [106]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [107]:
result = data.stack()

In [108]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [109]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Unstack a different level by passing a level number or name

In [110]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [111]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Unstacking might introduce missing data if all of the values in the level aren't found

In [113]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [114]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [115]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [116]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


Stacking filters out missing data by default

In [117]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [118]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

The level unstacked becomes the lowest level in the result

In [122]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                columns=pd.Index(['left', 'right'], name='side'))

In [123]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [124]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [125]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### 3.2 Pivoting "Long" to "Wide" Format

"Long" and "stacked" format

In [144]:
data = pd.read_csv('examples/macrodata.csv')

In [145]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [146]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')

In [147]:
data = data[['realgdp', 'infl', 'unemp']]

In [148]:
data.columns.name = 'item'

In [149]:
data.index = periods.to_timestamp('D', 'end')

In [150]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2


In [151]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [152]:
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [160]:
pivoted = ldata.pivot(index='date', columns='item', values='value')

In [161]:
pivoted[:10]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


Reshaping two value columns simultaneously

In [162]:
ldata['value2'] = np.random.randn(len(ldata))

In [163]:
ldata[:10]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,1.449636
1,1959-03-31,infl,0.000,-0.602251
2,1959-03-31,unemp,5.800,-0.002596
3,1959-06-30,realgdp,2778.801,-0.349293
4,1959-06-30,infl,2.340,-0.768281
5,1959-06-30,unemp,5.100,-0.200387
6,1959-09-30,realgdp,2775.488,0.592619
7,1959-09-30,infl,2.740,0.826464
8,1959-09-30,unemp,5.300,2.085521
9,1959-12-31,realgdp,2785.204,-0.442335


In [164]:
pivoted = ldata.pivot(index='date', columns='item')

In [165]:
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.602251  1.449636 -0.002596
1959-06-30  2.34  2778.801   5.1 -0.768281 -0.349293 -0.200387
1959-09-30  2.74  2775.488   5.3  0.826464  0.592619  2.085521
1959-12-31  0.27  2785.204   5.6 -0.550480 -0.442335  1.775874
1960-03-31  2.31  2847.699   5.2 -0.420201  0.156694  0.820634

__pivot is equivalent to creating a hierarchical index using set_index followed by a call to unstack__

In [168]:
ldata.set_index(['date', 'item']).unstack().head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.602251  1.449636 -0.002596
1959-06-30  2.34  2778.801   5.1 -0.768281 -0.349293 -0.200387
1959-09-30  2.74  2775.488   5.3  0.826464  0.592619  2.085521
1959-12-31  0.27  2785.204   5.6 -0.550480 -0.442335  1.775874
1960-03-31  2.31  2847.699   5.2 -0.420201  0.156694  0.820634

### 3.3 Pivoting "Wide" to "Long" Format

An inverse operation to `pivot` for DataFrames is `pandas.melt`, which merges multiple columns into one

In [169]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})

In [170]:
df

,A,B,C,key
0,1,4,7,foo
1,2,5,8,bar
2,3,6,9,baz


Using 'key' as the only group indicator

In [172]:
melted = pd.melt(df, id_vars=['key'])

In [173]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


Using pivot te reshape back to the original layout

In [178]:
reshaped = melted.pivot('key', 'variable', 'value')

In [179]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [180]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


Specifying only a subset of columns to use as value columns

In [182]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


`pandas.melt` can be used without any group identifiers too

In [183]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9
